In [102]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier

In [93]:
# 讀取酒類資料集
digits = datasets.load_wine()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=42)

# 建立模型
reg = GradientBoostingRegressor(random_state=7)


In [94]:
?GradientBoostingRegressor

In [95]:
# 先看看使用預設參數得到的結果，約為 0.14 的 MSE
reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

0.1436066329824748


## 使用GridSearchCV
## 建構環境
#### GradientBoostingRegressor
#### n_estimators : 決策樹的數量
#### max_depth : 每棵樹的限制深度
#### param_grid建立字典整格
#### GridSearchCV : 1.訓練器 2.參數字典 3.使用分數評估器 

In [96]:
# 設定要訓練的超參數組合
n_estimators = [100,200,300]
max_depth = [3,5,7]
learning_rate = [0.05,0.1,0.15]
random_state =[7,None]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth,learning_rate=learning_rate,random_state=random_state)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(reg, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1,iid=True)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 263 out of 270 | elapsed:    4.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:    4.4s finished
c:\users\user\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


## 印出最佳結果與最佳參數

In [97]:
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.053001 using {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'random_state': None}


In [98]:
grid_result.best_params_
print("['max_depth']",grid_result.best_params_['max_depth'])
print("['n_estimators']",grid_result.best_params_['n_estimators'])

['max_depth'] 5
['n_estimators'] 300


In [99]:
# 使用最佳參數重新建立模型
reg_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'],
                                          learning_rate =grid_result.best_params_['learning_rate'] ,
                                         random_state=grid_result.best_params_['random_state'])


# 訓練模型
reg_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = reg_bestparam.predict(x_test)

## 注意!
## 並非使用GridSearchCV就能達到最佳優化
## 可能會遇到當參數字典設定不夠全面的情況

In [100]:
# 調整參數後約可降至 0.05 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

0.17166730165697566


# 試驗另一組dataset

In [115]:
digits = datasets.load_digits()
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=4)
clf = GradientBoostingClassifier()

In [116]:
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred))

0.9688888888888889


In [117]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   53.8s finished


In [118]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.961391 using {'max_depth': 3, 'n_estimators': 200}


In [119]:
grid_result.best_params_

{'max_depth': 3, 'n_estimators': 200}

In [120]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [121]:
print(metrics.accuracy_score(y_test, y_pred))

0.9711111111111111
